In [1]:
from tensorflow import keras
from keras import layers, activations, Input
from keras.datasets import cifar10
from keras.utils import np_utils
import cv2
import numpy as np

In [2]:
def load_cifar10_data(img_rows, img_cols):
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    x_train = x_train[0:2000, :, :, :]
    y_train = y_train[0:2000]

    x_test = x_test[0:500, :, :, :]
    y_test = y_test[0:500]

    x_train = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_train[:, :, :, :]])
    x_test = np.array([cv2.resize(img, (img_rows, img_cols)) for img in x_test[:, :, :, :]])

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train = x_train / 255.0
    x_test = x_test / 255.0

    return x_train, y_train, x_test, y_test

x_train, y_train, x_test, y_test = load_cifar10_data(224, 224)
img_input = Input(shape=(224, 224, 3))

In [3]:
class ResidualUnit(layers.Layer):
    def __init__(self, filters, strides=1, activation='relu', **kwargs):
        super().__init__(**kwargs)
        self.activation = activations.get(activation)
        self.main_layers = [layers.Conv2D(filters, (3, 3),
             strides=strides, padding='same', use_bias=False),
            layers.BatchNormalization(),
            self.activation,
            layers.Conv2D(filters, (3, 3), strides=1, padding='same', use_bias=False),
            layers.BatchNormalization()]
        self.skip_layers = []
        if strides > 1:
            self.skip_layers = [
                layers.Conv2D(filters, (1, 1), strides=strides, padding = 'same', use_bias=False),
                layers.BatchNormalization()]

    def call(self, inputs):
        Z = inputs
        for layer in self.main_layers:
            Z = layer(Z)
        skip_Z = inputs
        for layer in self.skip_layers:
            skip_Z = layer(skip_Z)
        return self.activation(Z + skip_Z)

In [4]:
model = keras.models.Sequential()
model.add(layers.Conv2D(64, (7, 7), strides=(2, 2), input_shape=[224, 224, 3], padding='same',use_bias=False))
model.add(layers.BatchNormalization())
model.add(layers.Activation('relu'))
model.add(layers.MaxPool2D(pool_size=(3,3), strides=(2, 2), padding='same'))
prev_filters = 64
for filters in [64] * 3 + [128] * 4 + [256] * 6 + [512] * 3:
    strides = 1 if filters == prev_filters else 2
    model.add(ResidualUnit(filters,strides=strides))
    prev_filters = filters
model.add(layers.GlobalAvgPool2D())
model.add(layers.Flatten())
model.add(layers.Dense(10, activation='softmax'))

In [ ]:
model.compile(optimizer='adam',
             loss = 'sparse_categorical_crossentropy',
             metrics = ['accuracy'])
model.fit(x_train,y_train,epochs=5,validation_data=(x_test,y_test))

Epoch 1/5
16/63 [======>.......................] - ETA: 14:49 - loss: 3.8042 - accuracy: 0.1562

In [ ]:
score = model.evaluate(x_test, y_test)
print('테스트 정확도 :', score[1])